# Modelos Lineares


In [ ]:
# @title Lendo a planilha

import numpy as np
import pandas as pd

df = pd.read_csv("dados.csv", index_col=0)

ciclos = df.index.values
t_feed = df["t_feed"].values
t_rinse = df["t_rinse"].values
t_blow = df["t_blow"].values
t_purge = df["t_purge"].values

purity_H2 = df["purity_H2"].values
H2_CO_ratio = df["H2_CO_ratio"].values
purity_CO2 = df["purity_CO2"].values
recovery_CO2 = df["recovery_CO2"].values
productivity = df["productivity"].values


In [ ]:
# Colocando dados em desvio

y = np.array(
    [
        purity_H2,
        H2_CO_ratio,
        purity_CO2,
        recovery_CO2,
        productivity,
    ]
).T
u = np.array([t_feed, t_rinse, t_blow, t_purge]).T

# ciclo de referência
ref_idx = 1
y_ref = np.tile(y[ref_idx, :], (len(y), 1))
u_ref = np.tile(u[ref_idx, :], (len(u), 1))

# --- Calcular desvios ---
y_desvio = y - y_ref
u_desvio = u - u_ref


# Gerando modelos lineares


# ARX


In [ ]:
def gerar_ARX(y, u):
    # Construindo a matriz de regressão
    X = np.hstack([y[:-1, :], u[:-1, :]])

    # Saídas futuras
    Y = y[1:, :]

    # Estimando parâmetros via mínimos quadrados
    theta = np.linalg.lstsq(X, Y)[0]
    return theta


theta = gerar_ARX(y_desvio, u_desvio)

print("Parâmetros estimados:\n", theta)

# Simulando saída estimada
N = len(y_desvio)

y_arx = np.zeros_like(y_desvio)
y_arx[0] = y_desvio[0]  # inicializa com o primeiro valor real

for k in range(1, N):
    phi = np.concatenate([y_arx[k - 1], u_desvio[k - 1]])
    y_arx[k] = phi @ theta

y_arx = y_arx + y_ref  # Convertendo o valor desvio em variavel de engenharia


In [ ]:
# Plotando resultados
from lib.plot import plt

plt.figure()

plt.plot(ciclos, y[:, 0], label="Purity H2")
plt.scatter(ciclos, y_arx[:, 0], label="Purity H2 (ARX)")

plt.plot(ciclos, y[:, 1], label="H2/CO ratio")
plt.scatter(ciclos, y_arx[:, 1], label="H2/CO ratio (ARX)")

plt.plot(ciclos, y[:, 2], label="Purity CO2")
plt.scatter(ciclos, y_arx[:, 2], label="Purity CO2 (ARX)")

plt.plot(ciclos, y[:, 3], label="Recovery CO2")
plt.scatter(ciclos, y_arx[:, 3], label="Recovery CO2 (ARX)")

plt.plot(ciclos, y[:, 4], label="Productivity")
plt.scatter(ciclos, y_arx[:, 4], label="Productivity (ARX)")

plt.ylim(0, 200)

plt.xlabel("Ciclo")
plt.ylabel("Saída")
plt.legend()
plt.grid(True)
plt.show()


## Espaço de estados


In [ ]:
A = theta[:5, :].T
B = theta[5:, :].T
C = np.eye(5)
D = np.zeros((5, 4))

x = y_desvio[0].copy()  # inicializa com a primeira saída
y_ss = np.zeros_like(y_desvio)
y_ss[0] = x

for k in range(1, N):
    x = A @ x + B @ u_desvio[k - 1]
    y_ss[k] = C @ x + D @ u_desvio[k - 1]

y_ss = y_ss + y_ref  # Convertendo o valor desvio em variavel de engenharia


In [ ]:
# Plotando resultados
plt.figure()

plt.plot(ciclos, y[:, 0], label="Purity H2")
plt.scatter(ciclos, y_ss[:, 0], label="Purity H2 (Espaço de Estados)")

plt.plot(ciclos, y[:, 1], label="H2/CO ratio")
plt.scatter(ciclos, y_ss[:, 1], label="H2/CO ratio (Espaço de Estados)")

plt.plot(ciclos, y[:, 2], label="Purity CO2")
plt.scatter(ciclos, y_ss[:, 2], label="Purity CO2 (Espaço de Estados)")

plt.plot(ciclos, y[:, 3], label="Recovery CO2")
plt.scatter(ciclos, y_ss[:, 3], label="Recovery CO2 (Espaço de Estados)")

plt.plot(ciclos, y[:, 4], label="Productivity")
plt.scatter(ciclos, y_ss[:, 4], label="Productivity (Espaço de Estados)")

plt.xlabel("Ciclo")
plt.ylabel("Saída")
plt.legend()
plt.grid(True)
plt.close()  # plt.show()

print("A:", A)
print("B:", B)
print("C:", C)
print("D:", D)
